In [1]:
using Plots
using Plots.PlotMeasures
include("../coevolution_network_base.jl")
using .CoevolutionNetworkBase
using LaTeXStrings
using Printf

In [2]:
default(
    fontfamily="Computer Modern",
    # titlefont=16,         # Font size for title
    guidefont=28,         # Font size for axis labels
    tickfont=16,           # Font size for tick labels
    size=(1200, 900),
    # left_margin=20, right_margin=20, top_margin=20, bottom_margin=20
    )



In [ ]:
# Parameters
L = 40.0
dx = 0.1
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.5
alpha = 0.0
gamma = 1.0
D = 0.01
Nh = 2 * 10^6

# Initialize viral and immune densities
viral_density = zeros(Float64, length(x))
viral_density[Int(round(length(x)/2))] = 100/dx

# initial_variance = 0.1
# viral_density .= 100/sqrt(2 * pi * initial_variance) .* exp.(-x.^2/2/initial_variance)
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)

# Create Network instance
migration_matrix = [0.0][:,:]
println(size(migration_matrix,1))
network = Network([population], migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 40.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

# Run the simulation
@time run_simulation!(simulation)
println(length(simulation.duration_times))
p = plot(population.xs, population.viral_density, xlabel="test, x")
display(p)

In [ ]:
anim = Plots.@animate for i in 2:Int(round(length(simulation.duration_times)))
    # Create the primary plot
    p = plot(yscale=:log10, ylims=(1, Nh), xlims=(-L/2, L/2), xlabel=L"antigenic coordinate $x$", ylabel=L"infected density, $n(x,t)$", 
            yticks=[10^n for n in 0:floor(Int, log10(Nh))], size=(1200, 900), left_margin=5mm, right_margin=5mm, top_margin=5mm, bottom_margin=5mm)

    # Plot viral densities on the primary plot
    reg1 = simulation.trajectory[i].populations[1].viral_density .> 0
    plot!(p, population.xs[reg1], simulation.trajectory[i].populations[1].viral_density[reg1], linetype=:steppost, label=:none, lw=3)

    # Create and update the secondary y-axis for fitnesses
    p3 = twinx(p)

    plot!(p3, ylabel=L"fitness, $F(x)$", legend=:none)
    plot!(p3, simulation.trajectory[i].populations[1].xs, simulation.trajectory[i].populations[1].fitness, ylims=(-1, 1.5), lw=3, color=:olive)
    hline!(p3, [0], linestyle=:dash, color=:black, label=:none, lw=3)
end

mp4(anim, "one_deme_example.mp4", fps=60)


In [ ]:
anim = Plots.@animate for i in 2:Int(round(length(simulation.duration_times))) 
    # Calculate total infected numbers for the right plot
    total_infected = calculate_total_infected(simulation)

    # Create the left plot (same as original)
    p_left = plot(yscale=:log10, ylims=(1, Nh), xlims=(-L/2, L/2), xlabel=L"antigenic coordinate $x$", ylabel=L"infected density, $n(x,t)$", 
                  yticks=[10^n for n in 0:floor(Int, log10(Nh))], size=(1200, 900), left_margin=30mm, right_margin=25mm, top_margin=5mm, bottom_margin=25mm, labelpadding=2, framestyle=:box)

    # Plot viral densities on the primary plot
    reg1 = simulation.trajectory[i].populations[1].viral_density .> 0
    plot!(p_left, population.xs[reg1], simulation.trajectory[i].populations[1].viral_density[reg1], linetype=:steppost, label=:none, lw=3)

    # Create and update the secondary y-axis for fitnesses
    p3_left = twinx(p_left)

    # Separate the label and the time value
    label_str = "time:"
    time_value_str = @sprintf("%5.1f", simulation.duration_times[i])

    plot!(p3_left, ylabel=L"fitness, $F(x)$", legend=:none, tickpadding=2, labelpadding=2)
    plot!(p3_left, simulation.trajectory[i].populations[1].xs, simulation.trajectory[i].populations[1].fitness, ylims=(-1, 1.5), lw=3, color=:olive)
    hline!(p3_left, [0], linestyle=:dash, color=:black, label=:none, lw=3)

    # Create the right plot (total infected as a function of time)
    filt = (total_infected .> 0) .& (simulation.duration_times .<= simulation.duration_times[i]) 
    p_right = plot(simulation.duration_times[filt], total_infected[filt], xlabel=L"time, $t$", ylabel=L"total infected, $N(t)$", yscale=:log10, lw=3, color=:blue, legend=:none, 
    xlims=(simulation.duration_times[1], simulation.duration_times[end]), ylims=(1, 1e7), size=(1200, 900), left_margin=25mm, right_margin=5mm, top_margin=5mm, bottom_margin=25mm, labelpadding=2)

    # Combine left and right plots side by side with increased spacing
    p_combined = plot(p_left, p_right, layout=(1, 2), size=(2900, 1100))
end

mp4(anim, "two_panel_deme_example.mp4", fps=60)


Now we will do the same thing with escape 

In [ ]:
# Parameters
L = 40.0
dx = 0.1
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.5
alpha = 0.0
gamma = 1.0
D = 0.01
Nh = 2 * 10^6

# Initialize viral and immune densities
viral_density = zeros(Float64, length(x))
viral_density[Int(round(length(x)/2))] = 100/dx

# initial_variance = 0.1
# viral_density .= 100/sqrt(2 * pi * initial_variance) .* exp.(-x.^2/2/initial_variance)
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)

# Create Network instance
migration_matrix = [0.0][:,:]
println(size(migration_matrix,1))
network = Network([population], migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 60.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

# Run the simulation
@time run_simulation!(simulation)
println(length(simulation.duration_times))
p = plot(population.xs, population.viral_density, xlabel="test, x")
display(p)

In [ ]:
anim = Plots.@animate for i in 2:Int(round(length(simulation.duration_times))) 
    # Calculate total infected numbers for the right plot
    total_infected = calculate_total_infected(simulation)

    # Create the left plot (same as original)
    p_left = plot(yscale=:log10, ylims=(1, Nh), xlims=(-L/2, L/2), xlabel=L"antigenic coordinate $x$", ylabel=L"infected density, $n(x,t)$", 
                  yticks=[10^n for n in 0:floor(Int, log10(Nh))], size=(1200, 900), left_margin=30mm, right_margin=25mm, top_margin=5mm, bottom_margin=25mm, labelpadding=2, framestyle=:box)

    # Plot viral densities on the primary plot
    reg1 = simulation.trajectory[i].populations[1].viral_density .> 0
    plot!(p_left, population.xs[reg1], simulation.trajectory[i].populations[1].viral_density[reg1], linetype=:steppost, label=:none, lw=3)

    # Create and update the secondary y-axis for fitnesses
    p3_left = twinx(p_left)

    # Separate the label and the time value
    label_str = "time:"
    time_value_str = @sprintf("%5.1f", simulation.duration_times[i])

    plot!(p3_left, ylabel=L"fitness, $F(x)$", legend=:none, tickpadding=2, labelpadding=2)
    plot!(p3_left, simulation.trajectory[i].populations[1].xs, simulation.trajectory[i].populations[1].fitness, ylims=(-1, 1.5), lw=3, color=:olive)
    hline!(p3_left, [0], linestyle=:dash, color=:black, label=:none, lw=3)

    # Create the right plot (total infected as a function of time)
    filt = (total_infected .> 0) .& (simulation.duration_times .<= simulation.duration_times[i]) 
    p_right = plot(simulation.duration_times[filt], total_infected[filt], xlabel=L"time, $t$", ylabel=L"total infected, $N(t)$", yscale=:log10, lw=3, color=:blue, legend=:none, 
    xlims=(simulation.duration_times[1], simulation.duration_times[end]), ylims=(1, 1e7), size=(1200, 900), left_margin=25mm, right_margin=5mm, top_margin=5mm, bottom_margin=25mm, labelpadding=2)

    # Combine left and right plots side by side with increased spacing
    p_combined = plot(p_left, p_right, layout=(1, 2), size=(2900, 1100))
end

mp4(anim, "two_panel_one_deme_escape.mp4", fps=60)


In [ ]:
# Parameters
L = 40.0
dx = 0.1
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.5
alpha = 0.0
gamma = 1.0
D = 0.01
Nh = 2 * 10^6

# Initialize viral and immune densities
viral_density = zeros(Float64, length(x))
viral_density[Int(round(length(x)/2))] = 100/dx

initial_variance = 0.01
viral_density .= 100/sqrt(2 * pi * initial_variance) .* exp.(-x.^2/2/initial_variance)
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)

# Create Network instance
migration_matrix = [0.0][:,:]
println(size(migration_matrix,1))
network = Network([population], migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 80.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

# Run the simulation
@time run_simulation!(simulation)
println(length(simulation.duration_times))
p = plot(population.xs, population.viral_density, xlabel="test, x")
display(p)

In [ ]:
anim = Plots.@animate for i in 2:Int(round(length(simulation.duration_times)))
    # Create the primary plot
    p = plot(yscale=:log10, ylims=(1, Nh), xlims=(-L/2, L/2), xlabel=L"antigenic coordinate $x$", ylabel=L"infected density, $n(x,t)$", 
            yticks=[10^n for n in 0:floor(Int, log10(Nh))], size=(1200, 900), left_margin=5mm, right_margin=5mm, top_margin=5mm, bottom_margin=5mm)

    # Plot viral densities on the primary plot
    reg1 = simulation.trajectory[i].populations[1].viral_density .> 0
    plot!(p, population.xs[reg1], simulation.trajectory[i].populations[1].viral_density[reg1], linetype=:steppost, label=:none, lw=3)

    # Create and update the secondary y-axis for fitnesses
    p3 = twinx(p)

    plot!(p3, ylabel=L"fitness, $F(x)$", legend=:none)
    plot!(p3, simulation.trajectory[i].populations[1].xs, simulation.trajectory[i].populations[1].fitness, ylims=(-1, 1.5), lw=3, color=:olive)
    hline!(p3, [0], linestyle=:dash, color=:black, label=:none, lw=3)
end

mp4(anim, "one_deme_low_var.mp4", fps=60)


In [ ]:
# Parameters
L = 40.0
dx = 0.1
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.5
alpha = 0.0
gamma = 1.0
D = 0.01
Nh = 2 * 10^6

# Initialize viral and immune densities
viral_density = zeros(Float64, length(x))
viral_density[Int(round(length(x)/2))] = 100/dx

initial_variance = 0.2
viral_density .= 100/sqrt(2 * pi * initial_variance) .* exp.(-x.^2/2/initial_variance)
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)

# Create Network instance
migration_matrix = [0.0][:,:]
println(size(migration_matrix,1))
network = Network([population], migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 80.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

# Run the simulation
@time run_simulation!(simulation)
println(length(simulation.duration_times))
p = plot(population.xs, population.viral_density, xlabel="test, x")
display(p)

In [ ]:
anim = Plots.@animate for i in 2:Int(round(length(simulation.duration_times)))
    # Create the primary plot
    p = plot(yscale=:log10, ylims=(1, Nh), xlims=(-L/2, L/2), xlabel=L"antigenic coordinate $x$", ylabel=L"infected density, $n(x,t)$", 
            yticks=[10^n for n in 0:floor(Int, log10(Nh))], size=(1200, 900), left_margin=5mm, right_margin=5mm, top_margin=5mm, bottom_margin=5mm)

    # Plot viral densities on the primary plot
    reg1 = simulation.trajectory[i].populations[1].viral_density .> 0
    plot!(p, population.xs[reg1], simulation.trajectory[i].populations[1].viral_density[reg1], linetype=:steppost, label=:none, lw=3)

    # Create and update the secondary y-axis for fitnesses
    p3 = twinx(p)

    plot!(p3, ylabel=L"fitness, $F(x)$", legend=:none)
    plot!(p3, simulation.trajectory[i].populations[1].xs, simulation.trajectory[i].populations[1].fitness, ylims=(-1, 1.5), lw=3, color=:olive)
    hline!(p3, [0], linestyle=:dash, color=:black, label=:none, lw=3)
end

mp4(anim, "one_deme_high_var.mp4", fps=60)
